<a href="https://colab.research.google.com/github/jumafernandez/data-science-II/blob/main/Stacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Métodos de ensamble: Stacking

En esta notebook implementamos un pequeño ejemplo de utilización de métodos de ensamble mediante la estrategia de _stacking_, donde hay un conjunto de modelos independientes que aprenden de los datos y luego un _meta-modelo_ que aprende a partir de las predicciones de los modelos independientes, lo cual se ilustra en el siguiente esquema:

<img src="https://editor.analyticsvidhya.com/uploads/39725Stacking.png" alt="Esquema stacking general" width="500">


Cargamos las librerías necesarias:

In [1]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

Cargamos el dataset a utilizar, en nuestro caso el de cancer de mama y lo separamos en train y test:

In [2]:
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Entrenamos modelos base por separado y verificamos el accuracy:

In [7]:
# Modelos base
clf1 = DecisionTreeClassifier(random_state=42)
clf2 = SVC(probability=True, random_state=42)  # Es necesario probability=True para stacking
clf3 = KNeighborsClassifier(n_neighbors=3)
clf4 = XGBClassifier(eval_metric='logloss')

# Entrenamiento y evaluación individual de cada modelo
clf1.fit(X_train, y_train)
y_pred1 = clf1.predict(X_test)
print(f"Accuracy Decision Tree: {accuracy_score(y_test, y_pred1):.2f}")

clf2.fit(X_train, y_train)
y_pred2 = clf2.predict(X_test)
print(f"Accuracy SVM: {accuracy_score(y_test, y_pred2):.2f}")

clf3.fit(X_train, y_train)
y_pred3 = clf3.predict(X_test)
print(f"Accuracy KNN: {accuracy_score(y_test, y_pred3):.2f}")

clf4.fit(X_train, y_train)
y_pred4 = clf4.predict(X_test)
print(f"Accuracy XGBoost: {accuracy_score(y_test, y_pred3):.2f}")

Accuracy Decision Tree: 0.95
Accuracy SVM: 0.95
Accuracy KNN: 0.93
Accuracy XGBoost: 0.93


Ahora entrenamos el ensamble a través de la estrategia de stacking:

In [6]:
# Aplicación de Stacking
estimators = [
    ('decision_tree', DecisionTreeClassifier(random_state=42)),
    ('svm', SVC(probability=True, random_state=42)),
    ('knn', KNeighborsClassifier(n_neighbors=3)),
    ('xgboost', XGBClassifier(eval_metric='logloss'))  # Agregamos XGBoost
]

# Meta-modelo: Regresión Logística
stacking_clf = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(random_state=42)
)

# Entrenamiento del modelo Stacking
stacking_clf.fit(X_train, y_train)
y_pred_stack = stacking_clf.predict(X_test)

# Evaluación del modelo Stacking
print(f"Accuracy Stacking: {accuracy_score(y_test, y_pred_stack):.2f}")


Accuracy Stacking: 0.96
